### Imports

In [1]:
from PW_explorer.load_worlds import load_worlds
from PW_explorer.run_clingo import run_clingo

In [2]:
from DLV_Input_Parser.dlv_rules_parser import parse_dlv_rules

In [3]:
%load_ext PWE_NB_Extension

### Implementation

We know that the symmetry degree is defined as the number of automorphism groups of the incidence graph of a query. Below we define an encoding that finds all the automorphisms of a given graph.

##### Automorphism Encoding

In [4]:
%%clingo --donot-display_input -lci automorphisms --donot-run

% e(X,Y) :- e(Y,X). --> only if undirected
gnode(X) :- e(X,_).
gnode(X) :- e(_,X).

vmap(X,Y) ; vout(X,Y) :- gnode(X), gnode(Y).
:- vmap(X1,Y1), vmap(X2,Y2), e(X1,X2), not e(Y1,Y2).
:- vmap(X1,Y1), vmap(X2,Y2), not e(X1,X2), e(Y1,Y2).

% used1(X) :- vmap(X,_).
% :- gnode(X), not used1(X).
% :- vmap(X,Y),vmap(X,Z),Y!=Z.
% :- vmap(Y,X),vmap(Z,X),Y!=Z.

:- gnode(X), #count {Y: vmap(X,Y)} != 1.
:- gnode(X), #count {Y: vmap(Y,X)} != 1.

% #show vmap/2.
#show.

In [5]:
symm_degree_rules = str(automorphisms).split('\n')

We can construct a incidence graph of query Q using the function below.

In [6]:
def get_incidence_graph_edge_facts(rule):
    listener = parse_dlv_rules(rule, print_parse_tree=False)
    edges = []
    for rule in listener.rules:
        head_atoms, tail_atoms = rule[0], rule[1]
        atom_count = 0
        for head in head_atoms+tail_atoms:
            atom_node = '"{}_{}_{}"'.format(head.rel_name, head.rel_arity, atom_count)
            edges.extend([('"{}"'.format(v), atom_node) for v in head.vars])
            atom_count += 1
    edge_facts = []
    for e in edges:
        edge_facts.append("e({},{}).".format(*e))
    return edge_facts

We now combine these two to find the symmetry degree.

In [7]:
def get_symm_degree(rule):
    edge_facts = get_incidence_graph_edge_facts(rule)
#     print(edge_facts)
    asp_out, _ = run_clingo(symm_degree_rules+edge_facts)
    _, _, pw_objs = load_worlds(asp_out, silent=True)
    return len(pw_objs)

#### Testing

In [8]:
# tri/0
get_symm_degree('tri :- e(X,Y), e(Y,Z), e(Z,X).')

6

In [9]:
# tri/1
get_symm_degree('tri(X) :- e(X,Y), e(Y,Z), e(Z,X).')

2

In [10]:
# tri/2
get_symm_degree('tri(X,Y) :- e(X,Y), e(Y,Z), e(Z,X).')

4

In [11]:
# tri/3
get_symm_degree('tri(X,Y,Z) :- e(X,Y), e(Y,Z), e(Z,X).')

6

In [12]:
# thop/0
get_symm_degree('thop :- hop(X,Z1), hop(Z1,Z2), hop(Z2,Y).')

2

In [13]:
# thop/1
get_symm_degree('thop(X) :- hop(X,Z1), hop(Z1,Z2), hop(Z2,Y).')

1

In [14]:
# thop/2
get_symm_degree('thop(X,Y) :- hop(X,Z1), hop(Z1,Z2), hop(Z2,Y).')

8

In [15]:
# thop/3
get_symm_degree('thop(X,Y,Z1) :- hop(X,Z1), hop(Z1,Z2), hop(Z2,Y).')

1

In [16]:
# thop/4
get_symm_degree('thop(X,Y,Z1,Z2) :- hop(X,Z1), hop(Z1,Z2), hop(Z2,Y).')

2